In [31]:
import numpy as np 
import cv2

In [32]:
camera = 1
extrinsic_time = '00:00:56'

In [35]:
# import video 
cap = cv2.VideoCapture(f'../chessboard_videos/out{camera}F.mp4')

# get number of frames
amount_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
start_frame = 1000

# Set the frame skip interval (1 frame every 30)
frame_skip = 15

# save frames to array 
frames = []
gray_frames = []
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

for i in range(start_frame, amount_of_frames):  
    ret, frame = cap.read()
    if i % frame_skip == 0:        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        frames.append(frame)
        gray_frames.append(gray_frame)

cap.release()
cv2.destroyAllWindows() # destroy all opened windows

print(len(gray_frames))

177


In [36]:

# imS = cv2.resize(gray_frames[30], (1280, 720)) 
# cv2.imshow('test', imS)
# cv2.waitKey(0)

# cv2.destroyAllWindows()



In [37]:
# go through the frames and try to detect a checkerboard in them
corners_detected = {}  # frame_id, corners
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)  # todo: choose better params

chessboard_width = 5
chessboard_height = 7

# prepare object points
object_points = np.zeros((chessboard_width * chessboard_height, 3), np.float32)
object_points[:,:2] = np.mgrid[0:chessboard_height, 0:chessboard_width].T.reshape(-1,2) 

 
# Arrays to store object points and image points from all the images.
obj_points = [] # 3d point in real world space
img_points = [] # 2d points in image plane.

i = 0
while i < len(gray_frames):
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray_frames[i], (chessboard_height, chessboard_width), None)
    print(i, ret)
    
    if ret:
        # If found, add object points, image points (after refining them)
        obj_points.append(object_points)
        
        corners_refined = cv2.cornerSubPix(gray_frames[i], corners, (11,11), (-1,-1), criteria)  # todo: choose better params
        corners_detected[i] = corners_refined
        
        img_points.append(corners)
        
        i += 5
    else:
        i += 5  # skip the next five frames
    #if len(corners_detected) >= 50:
        #break

print(f'Number of frames where corners were detected: {len(corners_detected)}')

0 True
5 True
10 True
15 True
20 True
25 True
30 True
35 True
40 False
45 False
50 True
55 True
60 True
65 True
70 True
75 True
80 True
85 True
90 True
95 True
100 True
105 True
110 True
115 True
120 True
125 True
130 True
135 True
140 True
145 True
150 True
155 True
160 True
165 True
170 True
175 True
Number of frames where corners were detected: 34


In [38]:
# cv2.drawChessboardCorners(frames[10], (chessboard_height, chessboard_width), corners_detected[10], True)
# imS = cv2.resize(frames[10], (1280, 720)) 
# cv2.imshow('img', imS)
# cv2.waitKey(0)
 
# cv2.destroyAllWindows()


In [39]:
# calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray_frames[0].shape[::-1], None, None)
print(f'Camera Matrix: {mtx}')
print(f'Distortion: {dist}')

Camera Matrix: [[2.03425508e+03 0.00000000e+00 1.85906416e+03]
 [0.00000000e+00 2.05401046e+03 1.12199428e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion: [[-0.30915611  0.09657182  0.0009885   0.00158603 -0.01378949]]


In [40]:
# cut out single frame for extrinsic calibration
import moviepy.editor as mpy
maskclip = mpy.VideoFileClip(f'../videos/out{camera}.mp4')
maskclip.save_frame(f'../extrinsic_calibration_images/out{camera}.png', t=extrinsic_time)

In [41]:
# undistort an image - first refine camera matrix
img = cv2.imread(f'../extrinsic_calibration_images/out{camera}.png')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print(f'New camera matrix: {newcameramtx}')

# undistort - option 1
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst_1 = dst[y:y+h, x:x+w]

cv2.imwrite(f'../extrinsic_calibration_images/out{camera}_undistorted.png', dst_1)
cv2.destroyAllWindows()

New camera matrix: [[8.12358041e+02 0.00000000e+00 1.77871534e+03]
 [0.00000000e+00 8.23148286e+02 1.07761494e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [42]:
np.save(f'../intrinsic_matrices/{camera}_refined', newcameramtx)
np.save(f'../intrinsic_matrices/{camera}', mtx)
np.save(f'../distortions/{camera}', dist)